In [19]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
from utils import *

rootPath = "../../results/scenario/geothermal"


plt.style.use('bmh')
%matplotlib inline

nBase = pypsa.Network("../../results/base/base27Node2045/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nMean = pypsa.Network(rootPath + "-mean/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nOptimist = pypsa.Network(rootPath + "-optimist/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")

INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [38]:





def getDfValue(n):
  return [
    # totla
    n.statistics.capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_grouper").sum() / 1e9,
    # elec
    n.statistics.capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.optimal_capacity(groupby='de_generator_grouper').sum() / 1e3,
    n.statistics.supply(groupby='de_generator_grouper').sum() / 1e6,
    n.statistics.curtailment(groupby="de_generator_grouper").sum() / 1e6,
    n.statistics.optimal_capacity(groupby="de_import_elec_grouper").sum() / 1e3,
    n.statistics.supply(groupby="de_import_elec_grouper").sum() / 1e6,
    n.statistics.optimal_capacity(groupby="de_export_elec_grouper").sum() / 1e3,
    n.statistics.supply(groupby="de_export_elec_grouper").sum() / 1e6,
    # warm
    n.statistics.capex(groupby="de_heat_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_heat_generator_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_heat_generator_grouper").sum() / 1e9,
    n.statistics.optimal_capacity(groupby='de_heat_generator_grouper').sum() / 1e3,
    n.statistics.supply(groupby="de_heat_generator_grouper").sum() / 1e6,
    # Geothermie
    n.statistics.supply(groupby=["carrier", "de_generator_grouper"]).groupby('carrier').sum().get('geothermal organic rankine cycle', 0),
    renameHeatCarrier(n.statistics.supply(groupby=["carrier", "de_heat_generator_grouper"]).groupby('carrier').sum()).get('Geothermie', 0)
  ]

data = {
    'Name': [
       # total
      'CAPEX', 
      'investierte CAPEX', 
      'OPEX',
       # elec
      'Erzeugung CAPEX',
      'Erzeugung investierte CAPEX',
      'Erzeugung OPEX',
      'Stromkapazität',
      'Stromerzeugung',
      'curtailment',
      'Importkapazität',
      'Importmenge',
      'Exportkapazität',
      'Exportmenge',
       # heat
       'Wärm CAPEX',
       'Wärm investierte CAPEX',
       'Wärm OPEX',
       'Wärmekapazität',
       'Wärmerzeugung',
       # Geothermie
       'geothermal elec menge',
       'geothermal Wärm menge'
      ],
    'Unit': [
      # total
      'Mrd.Euro',
      'Mrd.Euro',
      'Mrd.Euro',
       # elec
      'Mrd.Euro',
      'Mrd.Euro',
      'Mrd.Euro',
      'GW',
      'TWH',
      'TWH',
      'GW',
      'TWH',
      'GW',
      'TWH',
      # heat
      'Mrd.Euro',
      'Mrd.Euro',
      'Mrd.Euro',
      'GW',
      'TWH',
      # Geothermie
      'TWH',
      'TWH',
      

    ],
    'Base': getDfValue(nBase),
    'Mean': getDfValue(nMean),
    'Optimist': getDfValue(nOptimist),
}

print(len(data['Name']), len(data['Unit']), len(data['Base']))

df = pd.DataFrame(data).set_index('Name')

print(df)

20 20 20
                                 Unit      Base      Mean  Optimist
Name                                                               
CAPEX                        Mrd.Euro  64.25519  64.25820  64.25875
investierte CAPEX            Mrd.Euro  44.91728  44.92022  44.92078
OPEX                         Mrd.Euro   6.63091   6.66073   6.64124
Erzeugung CAPEX              Mrd.Euro  26.20495  26.20595  26.20602
Erzeugung investierte CAPEX  Mrd.Euro  10.90579  10.90679  10.90686
Erzeugung OPEX               Mrd.Euro  10.90579  10.90679  10.90686
Stromkapazität                     GW 210.52356 210.54118 210.54232
Stromerzeugung                    TWH 589.77387 592.88136 592.52185
curtailment                       TWH   7.86180   4.77104   5.13165
Importkapazität                    GW  92.96970  92.96251  92.96505
Importmenge                       TWH 269.29716 269.31023 269.27131
Exportkapazität                    GW  98.19332  98.20198  98.20015
Exportmenge                       TWH 2